In [1]:

# this is a zip object in binary that starts PK 
# response.t

In [49]:
txt = """
State_abr	State_full		#_Records	
ak			Alaska			289956		
al			Alabama			1547626		
ar			Arkansas			1012208		
az			Arizona			1245436		
ca			California		5974018		
co			Colorado			1149525		
ct			Connecticut		625588		
dc			Washington D.C.		134979		
de			Delaware			196826		
fl			Florida			3886760		
ga			Georgia			2456506		
hi			Hawaii			270950		
ia			Iowa			882883		
id			Idaho			411877		
il			Illinois			2542348		
in			Indiana			1452459		
ks			Kansas			742178		
ky			Kentucky			1245393		
la			Louisiana		1256793		
ma			Massachusetts		1176936		
md			Maryland			921448		
me			Maine			549642		
mi			Michigan			1845096		
mn			Minnesota		1193806		
mo			Missouri			1409298		
ms			Mississippi		916881		
mt			Montana			416879		
nc			North Carolina		2544765		
nd			North Dakota		275411		
ne			Nebraska			544468		
nh			New Hampshire		384665		
nj			New Jersey		1294191		
nm			New Mexico		635024		
nv			Nevada			528668		
ny			New York			3219353		
oh			Ohio			2244990		
ok			Oklahoma			1218486		
or			Oregon			857899		
pa			Pennsylvania		2657972		
pr			Puerto Rico		443027		
ri			Rhode Island		175850		
sc			South Carolina		1191805		
sd			South Dakota		290266		
tn			Tennessee		1601759		
tx			Texas			5590579		
ut			Utah			564109		
va			Virginia			1777779		
vt			Vermont			306642		
wa			Washington		1405158		
wi			Wisconsin		1262316		
wv			West Virginia		815903		
wy			Wyoming			216994		
"""
state_codes = [line[0:2].upper() for line in txt.split("\n")[2::]]

pd.DataFrame({"state":state_codes}).to_csv("state_codes.csv", index=False)

In [34]:
import requests

cookies = {
    '_ga': 'GA1.2.409232890.1655852538',
    '_gid': 'GA1.2.23741594.1655852538',
    'uw_madison_cookieconsent_timestamp': '1655852540',
    'uw_madison_cookieconsent_url': 'https://www.neighborhoodatlas.medicine.wisc.edu/',
    '_gat_gtag_UA_57610822_10': '1',
    'sess': 's%3AIvZ_UtuD_kICmB1QKdDoV0L3ILO0L-IE.Hae9Ng5YAs7kpOXy3o3bJwufQspdpUezCFwp%2FCUCEug',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': '_ga=GA1.2.409232890.1655852538; _gid=GA1.2.23741594.1655852538; uw_madison_cookieconsent_timestamp=1655852540; uw_madison_cookieconsent_url=https://www.neighborhoodatlas.medicine.wisc.edu/; _gat_gtag_UA_57610822_10=1; sess=s%3AIvZ_UtuD_kICmB1QKdDoV0L3ILO0L-IE.Hae9Ng5YAs7kpOXy3o3bJwufQspdpUezCFwp%2FCUCEug',
    'Origin': 'https://www.neighborhoodatlas.medicine.wisc.edu',
    'Referer': 'https://www.neighborhoodatlas.medicine.wisc.edu/download',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

def _fetch_data(state_code: str = "AL") -> BytesIO:

    data = {
        'state-type': 'zipcode',
        '_csrf': 'rFxUqvF1-oRq_vCNNx0ObelwfQsMiia9gJx8',
        'scale-group': 'state',
        'state-name': state_code,
        'version-group': '19',
    }

    response = requests.post('https://www.neighborhoodatlas.medicine.wisc.edu/adi-download', cookies=cookies, headers=headers, data=data)
    return BytesIO(response.content)

In [50]:
from io import BytesIO, StringIO
from zipfile import ZipFile

def build_adi_table(state_code: str) -> None:
    zf = ZipFile(_fetch_data(state_code))
    content = StringIO(zf.read(zf.namelist()[0]).decode())
    df = pd.read_csv(content, dtype="str")
    df.to_parquet(f"tables/{state_code}.parquet")

for state in state_codes:
    build_adi_table(state)